In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [865 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,092 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,231 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ub

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [ ]:
df.dtypes

[('id', 'string'),
 ('date', 'string'),
 ('date_built', 'string'),
 ('price', 'string'),
 ('bedrooms', 'string'),
 ('bathrooms', 'string'),
 ('sqft_living', 'string'),
 ('sqft_lot', 'string'),
 ('floors', 'string'),
 ('waterfront', 'string'),
 ('view', 'string')]

In [ ]:
# 2. Create a temporary view of the DataFrame.
# Convert 'date' to date format
df.createOrReplaceTempView('home_sales')
spark.sql("select to_date('date','yyyy-mm-dd')")


DataFrame[to_date(date, yyyy-mm-dd): date]

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
select
round(avg(price),2),
year(date)
from home_sales
where bedrooms == 4
group by year(date)
sort by year(date) desc
""").show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|           301819.44|      2021|
|           298353.78|      2020|
|            300263.7|      2019|
+--------------------+----------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
select
round(avg(price),2),
date_built
from home_sales
where bedrooms == 3 and bathrooms == 3
group by date_built
sort by date_built desc
""").show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           292676.79|      2017|
|           290555.07|      2016|
|            288770.3|      2015|
|           290852.27|      2014|
|           295962.27|      2013|
|           293683.19|      2012|
|           291117.47|      2011|
|           292859.62|      2010|
+--------------------+----------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
select
round(avg(price),2),
date_built
from home_sales
where bedrooms == 3 and bathrooms == 3
and floors == 2 and sqft_living >= 2000
group by date_built
sort by date_built desc
""").show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           280317.58|      2017|
|            293965.1|      2016|
|           297609.97|      2015|
|           298264.72|      2014|
|           303676.79|      2013|
|           307539.97|      2012|
|           276553.81|      2011|
|           285010.22|      2010|
+--------------------+----------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
select
round(avg(price),2),
view
from home_sales
where price >= 350000
group by view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           403005.77|   7|
|           788128.21|  51|
|            404673.3|  15|
|           798684.82|  54|
|           399548.12|  11|
|           397771.65|  29|
|           750537.94|  69|
|            396964.5|  42|
|           1072285.2|  87|
|           752861.18|  73|
|           767036.67|  64|
|            398867.6|   3|
|            397862.0|  30|
|           401419.75|  34|
|            791453.0|  59|
|           398592.71|   8|
|           402124.62|  28|
|           402022.68|  22|
|          1056336.74|  85|
|           399586.53|  16|
+--------------------+----+
only showing top 20 rows

--- 1.12528395652771 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
select
round(avg(price),2),
view
from home_sales
where price >= 350000
group by view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           403005.77|   7|
|           788128.21|  51|
|            404673.3|  15|
|           798684.82|  54|
|           399548.12|  11|
|           397771.65|  29|
|           750537.94|  69|
|            396964.5|  42|
|           1072285.2|  87|
|           752861.18|  73|
|           767036.67|  64|
|            398867.6|   3|
|            397862.0|  30|
|           401419.75|  34|
|            791453.0|  59|
|           398592.71|   8|
|           402124.62|  28|
|           402022.68|  22|
|          1056336.74|  85|
|           399586.53|  16|
+--------------------+----+
only showing top 20 rows

--- 0.5453720092773438 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partition")

In [ ]:
# 11. Read the parquet formatted data.
df_p=spark.read.parquet('home_sales_partition')

In [ ]:
# 12. Create a temporary table for the parquet data.
df_p.createOrReplaceTempView('home_sales_p')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
select
round(avg(price),2),
view
from home_sales_p
where price >= 350000
group by view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           403005.77|   7|
|           788128.21|  51|
|            404673.3|  15|
|           798684.82|  54|
|           399548.12|  11|
|           397771.65|  29|
|           750537.94|  69|
|            396964.5|  42|
|           752861.18|  73|
|           1072285.2|  87|
|           767036.67|  64|
|            398867.6|   3|
|            397862.0|  30|
|           401419.75|  34|
|            791453.0|  59|
|           398592.71|   8|
|           402124.62|  28|
|           402022.68|  22|
|          1056336.74|  85|
|           401934.21|  35|
+--------------------+----+
only showing top 20 rows

--- 0.9537832736968994 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False